In [1]:
import netCDF4 as nc
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import pickle



In [2]:
name_data = 'discharge'

# -----------------------------------------------------------------------------  

# See data from station
info_stations = pd.read_csv('../Documents/porto_alegre_stations_wwhoutid.csv')

# Specify the directory path
hist_folder_path = f'Historic_{name_data}'
telem_folder_path = 'Telemetricas'

# for index, code in enumerate(info_stations['Code']):

# Set station of interest manually
index_ = 6

station_name = info_stations['Name'][index_]
station_code = info_stations['Code'][index_]
print('----------------------------------------------------------------------------')
print(station_code)
print(station_name)

    

----------------------------------------------------------------------------
85900000
RIO PARDO


In [3]:

def print_netcdf_summary(file_path, return_variables=None):

    print(return_variables)
    """Prints a summary and optionally returns data for specific variables from a NetCDF file."""
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    try:
        dataset = nc.Dataset(file_path, mode='r')
    except Exception as e:
        print(f"Error opening file: {e}")
        return

    variable_data = {}

    print("Global Attributes:")
    for attr in dataset.ncattrs():
        print(f"  {attr}: {dataset.getncattr(attr)}")

    print("\nDimensions:")
    for dim, dim_obj in dataset.dimensions.items():
        print(f"  {dim}: length {dim_obj.size} (unlimited: {dim_obj.isunlimited()})")

    print("\nVariables:")
    for var in dataset.variables:
        print(f"Variable: {var}")
        print(f"  Type: {dataset.variables[var].dtype}")
        print(f"  Dimensions: {dataset.variables[var].dimensions}")
        print(f"  Shape: {dataset.variables[var].shape}")
        for attr in dataset.variables[var].ncattrs():
            print(f"    {attr}: {dataset.variables[var].getncattr(attr)}")
        if return_variables and var in return_variables:
            variable_data[var] = dataset.variables[var][:]

    if 'time' in dataset.variables:
        T_var = dataset.variables['time']
        T_units = T_var.units if 'units' in T_var.ncattrs() else 'No units available'
        T_data = T_var[:]
        print(f"\nT units: {T_units} Data: {T_data}")

    dataset.close()

    if return_variables:
        print('retorna!')
        return variable_data


In [4]:

def read_station(folder_path, file_name):
    
    full_path = os.path.join(folder_path, f'{file_name}.txt')
    try:
        # Load observation data from a text file
        # Specify -99999 and -1 as NaN values
        df = pd.read_table(full_path, delim_whitespace=True, na_values=[-99999, -1])

        df['date'] = pd.to_datetime(df['date'])

    except FileNotFoundError:
        print(f"No data: {full_path}")
        return pd.DataFrame()  # Return empty DataFrame if file is not found
    
    return df


In [5]:

def list_files_and_dates(directory_path):
    """Checks if the directory exists and returns a DataFrame containing filenames and their corresponding dates."""
    if os.path.exists(directory_path):
        file_list = os.listdir(directory_path)
        file_dates = [file.split('_')[-1].split('.')[0] for file in file_list]
        file_dates = [datetime.strptime(date, '%Y%m%d%H') for date in file_dates]
        file_data = pd.DataFrame({
            'File Name': file_list,
            'Date': file_dates
        }).sort_values(by='Date').reset_index(drop=True)
        return file_data
    else:
        print(f"The directory '{directory_path}' does not exist.")
        return None
    

In [6]:

def process_glofas_data(file_data, station_code):
    
    forecast_dict = {}  # Use a dictionary to store the data by issue_date

    # Iterate over the forecast files
    for forecast in range(len(file_data)):
        filename = file_data['File Name'].loc[forecast]
        file_path = os.path.join('glofas_v4.0_forecasts_20240505-20240510/', filename)
        
        # Open the dataset
        dataset = nc.Dataset(file_path, mode='r')
        
        # Extract necessary variables from the dataset
        variables = {var: dataset.variables[var][:] for var in dataset.variables}
        ensemble = variables['ensemble']
        dis24 = variables['dis24']
        
        station_index = int(np.where(variables['code'] == station_code)[0])

        issue_date = file_data['Date'].loc[forecast]
        
        # Ensure issue_date is aware of timezone
        if issue_date.tzinfo is None or issue_date.tzinfo.utcoffset(issue_date) is None:
            issue_date = issue_date.replace(tzinfo=timezone.utc)
        
        start_date = issue_date + timedelta(days=0.5)  # Adjust to 12:00
        
        # Close the dataset
        dataset.close()

        # Extract data for the station
        dis24_station = dis24[:, station_index, :]  # dShape: (30, 11, 51)
        
        print(dis24_station)
        dis24_mean = np.mean(dis24_station, axis=1)  # Daily mean

        # Create a daily time series starting from the start date
        time = [start_date + timedelta(days=i) for i in range(30)]  # Assumes 30 days of data
        
        # Store data in the dictionary using the issue_date as the key
        forecast_dict[issue_date] = {
            'time': time,
            'dis24_ensem': dis24_station,
            'dis24_station': dis24_mean,
            'start_date': start_date
        }

    return forecast_dict, station_index


In [7]:

def plot_glofas_forecasts(forecast_dict, df_obs, station_name, threshold_data, name_data, glofas_index):
    
    f = 0

    # Iterate over the forecasts stored in the dictionary
    for issue_date, issue in forecast_dict.items():
        f += 1
        time = pd.to_datetime(issue['time'])  # Convert to datetime

        dis24_station = issue['dis24_ensem']
        dis24_mean = issue['dis24_station']
        issue_date = pd.to_datetime(issue_date)  # Convert issue_date to datetime if it's not already

        plt.figure(figsize=(8, 5))
        for i in range(dis24_station.shape[1]):
            plt.plot(time, dis24_station[:, i], color='grey', alpha=0.2)

        plt.plot(time, dis24_mean, color='blue', linewidth=2, alpha=0.8, label='Ensemble Mean')
        
        # Normalize the observed data 'date' column to midnight
        df_obs['date'] = pd.to_datetime(df_obs['date']).dt.normalize()

        plt.plot(df_obs['date'], df_obs[name_data], color='black', label='Observed Data')
        
        plt.xlabel('Date (2024)')
        plt.ylabel('Discharge (m³/s)')
        # plt.title(f'GloFAS 24-hour Discharge Forecast for {station_name} Station')
        plt.xlim(time[0] - timedelta(days=5), time[-1])
        plt.ylim(0,30000)
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        plt.gcf().autofmt_xdate()
        plt.grid(True, color='grey', alpha=0.2, which='both', linestyle='-', axis='x')

        plt.tight_layout()
        
        plt.axvline(x=issue_date, color='black', linestyle='--', label='Date of issue')
        
        # Draw threshold lines
        for key, value in threshold_data.items():
            plt.axhline(y=value[glofas_index], color='red' if key == 'rl50' else 'orange', linewidth=2, alpha=0.4, label=f'{key.upper()} Threshold')

        plt.legend(loc='upper right')
        plt.savefig(f'Figures/{station_name}_GloFAS_{f}.png', dpi=300)
        plt.show()


In [8]:
# See files from system
directory_path = 'glofas_v4.0_forecasts_20240505-20240510/'
file_data = list_files_and_dates(directory_path)
if file_data is not None:
    print(file_data)

                              File Name       Date
0   glofas_v4.0_forecasts_2024041500.nc 2024-04-15
1   glofas_v4.0_forecasts_2024041600.nc 2024-04-16
2   glofas_v4.0_forecasts_2024041700.nc 2024-04-17
3   glofas_v4.0_forecasts_2024041800.nc 2024-04-18
4   glofas_v4.0_forecasts_2024041900.nc 2024-04-19
5   glofas_v4.0_forecasts_2024042000.nc 2024-04-20
6   glofas_v4.0_forecasts_2024042100.nc 2024-04-21
7   glofas_v4.0_forecasts_2024042200.nc 2024-04-22
8   glofas_v4.0_forecasts_2024042300.nc 2024-04-23
9   glofas_v4.0_forecasts_2024042400.nc 2024-04-24
10  glofas_v4.0_forecasts_2024042500.nc 2024-04-25
11  glofas_v4.0_forecasts_2024042600.nc 2024-04-26
12  glofas_v4.0_forecasts_2024042700.nc 2024-04-27
13  glofas_v4.0_forecasts_2024042800.nc 2024-04-28
14  glofas_v4.0_forecasts_2024042900.nc 2024-04-29
15  glofas_v4.0_forecasts_2024043000.nc 2024-04-30
16  glofas_v4.0_forecasts_2024050100.nc 2024-05-01
17  glofas_v4.0_forecasts_2024050200.nc 2024-05-02
18  glofas_v4.0_forecasts_20240

In [9]:

filename = file_data['File Name'].loc[0]
file_path = os.path.join(directory_path, filename)
print_netcdf_summary(file_path)


None
Global Attributes:

Dimensions:
  stats: length 11 (unlimited: False)
  ensemble: length 51 (unlimited: False)
  time: length 30 (unlimited: False)
  nchar: length 200 (unlimited: False)

Variables:
Variable: time
  Type: float64
  Dimensions: ('time',)
  Shape: (30,)
    long_name: time
    standard_name: time
    calendar: standard
    axis: T
    units: hours since 2024-04-04 00:00:00
Variable: ensemble
  Type: int16
  Dimensions: ('ensemble',)
  Shape: (51,)
    units: -
    long_name: ensemble members
Variable: priority
  Type: int16
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Priority rank
Variable: name
  Type: <class 'str'>
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Name of the station
Variable: river
  Type: <class 'str'>
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Name of the river
Variable: code
  Type: int32
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Station code
Variable:

In [10]:
# See general caracteristics of some nectdf
file_path = 'thresholds.nc'

threshold_vars = ['rl1.5', 'rl50']
threshold_data = print_netcdf_summary(file_path, return_variables=threshold_vars)

['rl1.5', 'rl50']
Global Attributes:

Dimensions:
  stats: length 11 (unlimited: False)
  nchar: length 200 (unlimited: False)

Variables:
Variable: priority
  Type: int16
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Priority rank
Variable: name
  Type: <class 'str'>
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Name of the station
Variable: river
  Type: <class 'str'>
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Name of the river
Variable: code
  Type: int32
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Station code
Variable: lat
  Type: float32
  Dimensions: ('stats',)
  Shape: (11,)
    units: degrees_north
    long_name: latitude
Variable: lon
  Type: float32
  Dimensions: ('stats',)
  Shape: (11,)
    units: degrees_north
    long_name: longitude
Variable: city
  Type: <class 'str'>
  Dimensions: ('stats',)
  Shape: (11,)
    units: -
    long_name: Name of the city
Variable: glon
  Type: flo

In [11]:

name_data = 'discharge'

# -----------------------------------------------------------------------------
# Read data from station
info_stations = pd.read_csv('../Documents/porto_alegre_stations_wwhoutid.csv')

# Specify the directory paths
hist_folder_path = f'Historic_{name_data}'
telem_folder_path = 'Telemetricas'

# Loop through each station using the 'Code' column
for index, row in info_stations.iterrows():
    station_name = row['Name']
    station_code = row['Code']
    print('----------------------------------------------------------------------------')
    print(station_code)
    print(station_name)

    if threshold_data:
        file_data = list_files_and_dates('glofas_v4.0_forecasts_20240505-20240510/')
        df_telem = read_station('../Telemetry', station_code)

        processed_data, glofas_index = process_glofas_data(file_data, station_code)

        # Save the processed data to a file
        with open(f'data_glofas_{station_code}.pkl', 'wb') as file:
            pickle.dump(processed_data, file)

        # Plot the forecasts
        plot_glofas_forecasts(processed_data, df_telem, station_name, threshold_data, name_data, glofas_index)


----------------------------------------------------------------------------
87450004
CAIS MAUÁ C6


C:\Users\ingri\AppData\Local\Temp\ipykernel_51036\2608100217.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_table is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_table(full_path, delim_whitespace=True, na_values=[-99999, -1])
C:\Users\ingri\AppData\Local\Temp\ipykernel_51036\1659782673.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  station_index = int(np.where(variables['code'] == station_code)[0])


AttributeError: 'MaskedArray' object has no attribute 'head'